<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/catdog/Run_CatDog_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, print_function
!rm -r abstention
!git clone https://github.com/blindauth/abstention.git
!pip uninstall abstention
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-t6ja1k3u/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 1049, done.
remote: Counting objects: 100% (1049/1049), done.
remote: Compressing objects: 100% (1047/1047), done.
remote: Total 3987 (delta 6), reused 1040 (delta 2), pack-reused 2938
Receiving objects: 100% (3987/3987), 288.99 MiB | 40.84 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [0]:
import gzip
import numpy as np

def read_vals(gzip_file, idx_to_load=0):
    return np.array([[
        float(y) for y in x.decode("utf-8").rstrip().split("\t")][idx_to_load]
                          for x in gzip.open(gzip_file)])
  
def get_calc_tpr_at_fpr_func(fpr_level): 
    def tpr_at_fpr(y_true, y_score):
      #verify probs are sorted, if not, sort them
      if (np.min(y_score[1:] - y_score[:-1]) < 0):
          (sorted_y_score, sorted_y_true) = zip(*sorted(zip(y_score,y_true),
                                                   key=lambda x: x[0]))
          sorted_y_score = np.array(sorted_y_score)
          sorted_y_true = np.array(sorted_y_true)
      else:
          sorted_probs = y_score
          sorted_y_true = y_true
      pos_cumsum = np.cumsum(sorted_y_true)
      tprs = (pos_cumsum[-1]-pos_cumsum)/pos_cumsum[-1]
      neg_cumsum = np.cumsum(1-sorted_y_true)
      fprs = (neg_cumsum[-1]-neg_cumsum)/neg_cumsum[-1]
      for (tpr,fpr) in zip(tprs,fprs):
          if (fpr <= fpr_level):
              return tpr    
    return tpr_at_fpr

In [3]:
import numpy as np
import random
from abstention.abstention import (
    MonteCarloWindowAbstDeltaTprAtFprThreshold,
    EstWindowAbstDeltaAuroc,
    Uncertainty, NegPosteriorDistanceFromThreshold,
    FixedThreshold, OneMinusJSDivFromClassFreq,
    DualThresholdsFromPointFiveOnValidSet)
from abstention.calibration import PlattScaling
from abstention.label_shift import EMImbalanceAdapter, NoAdaptation
from sklearn.metrics import roc_auc_score
from collections import OrderedDict
import sys
import json
import os


model_dirs = [
  "record_1_model_wqD9U_preds",
  "record_2_model_eDiQm_preds",
  "record_3_model_Mwl4q_preds",
  "record_4_model_3Y6wQ_preds",
  "record_5_model_3L3uV_preds",
  "record_6_model_jXjNL_preds",
  "record_7_model_A5Cuf_preds",
  "record_9_model_QCnbs_preds",
  "record_10_model_Xu18x_preds",
  "record_11_model_O6Kxa_preds"]

target_fpr_level = 0.01
n_samples=100
smoothing_window_size=11

orig_test_labels = read_vals(
    "abstention_experiments/catdog/test_labels.txt.gz")
orig_valid_labels = read_vals(
    "abstention_experiments/catdog/valid_labels.txt.gz")

bootstrap_test_set_size = len(orig_valid_labels) #bootstrapped to be this big


metricname_to_computefunc = OrderedDict([
    ('tpratfpr_'+str(target_fpr_level),
     get_calc_tpr_at_fpr_func(target_fpr_level)),
    ('auroc', roc_auc_score)
])

abstname_to_factory = OrderedDict([
    ('mcwindowdelta_tpratfpr_30pc',
     MonteCarloWindowAbstDeltaTprAtFprThreshold(
         n_samples=n_samples, fpr_threshold=target_fpr_level,
         num_to_abstain_on=int(0.3*bootstrap_test_set_size),
         smoothing_window_size=smoothing_window_size,
         return_max_across_windows=True)),
    ('fumera_tpratfpr_30pc',
     DualThresholdsFromPointFiveOnValidSet(
         fracs_to_abstain_on=[0.3],
         metric=get_calc_tpr_at_fpr_func(target_fpr_level))),
    ('mcwindowdelta_tpratfpr_15pc',
     MonteCarloWindowAbstDeltaTprAtFprThreshold(
         n_samples=n_samples, fpr_threshold=target_fpr_level,
         num_to_abstain_on=int(0.15*bootstrap_test_set_size),
         smoothing_window_size=smoothing_window_size,
         return_max_across_windows=True)),
    ('fumera_tpratfpr_15pc',
     DualThresholdsFromPointFiveOnValidSet(
         fracs_to_abstain_on=[0.15],
         metric=get_calc_tpr_at_fpr_func(target_fpr_level))),   
    ('js', OneMinusJSDivFromClassFreq()),
    ('estwindowdelta_auroc_30pc',
     EstWindowAbstDeltaAuroc(
         num_to_abstain_on=int(0.3*bootstrap_test_set_size),
         return_max_across_windows=True)),
    ('estwindowdelta_auroc_15pc',
     EstWindowAbstDeltaAuroc(
         num_to_abstain_on=int(0.15*bootstrap_test_set_size),
         return_max_across_windows=True)),
    ('fumera_auroc_30pc', DualThresholdsFromPointFiveOnValidSet(
                           fracs_to_abstain_on=[0.3],
                           metric=roc_auc_score)),
    ('fumera_auroc_15pc', DualThresholdsFromPointFiveOnValidSet(
                          fracs_to_abstain_on=[0.15],
                          metric=roc_auc_score)),
    ('uncertainty', Uncertainty()),
    ('distfrom0.5', NegPosteriorDistanceFromThreshold(FixedThreshold(0.5))),  
])

calibfactory = PlattScaling(verbose=False)
adaptname_to_factory = OrderedDict([
    ('EM', EMImbalanceAdapter(verbose=False)),
    ('None', NoAdaptation())])

abstfracs = [0.15, 0.3]

num_trials_per_model = 3

###
#initialize everything
metricname_to_adaptname_to_baselineperfs = OrderedDict()
for metricname in metricname_to_computefunc:
  adaptname_to_baselineperfs = OrderedDict()
  for adaptname in adaptname_to_factory:
    adaptname_to_baselineperfs[adaptname] = []
  metricname_to_adaptname_to_baselineperfs[metricname] =(
    adaptname_to_baselineperfs)
  
metricname_to_abstfrac_to_abstnadapt_to_perf = OrderedDict()
for metricname in metricname_to_computefunc:
  abstfrac_to_abstnadapt_to_perf = OrderedDict()
  for abstfrac in abstfracs:
    abstnadapt_to_perf = OrderedDict()
    for abstname in abstname_to_factory:
      for adaptname in adaptname_to_factory:
        abstnadapt_to_perf[abstname+":"+adaptname] = []
    abstfrac_to_abstnadapt_to_perf[abstfrac] = abstnadapt_to_perf
  metricname_to_abstfrac_to_abstnadapt_to_perf[metricname] = (
    abstfrac_to_abstnadapt_to_perf)
  
####
for model_dir_idx,model_dir in list(enumerate(model_dirs))[:]:
  print("\n###On model dir",model_dir)
  orig_test_detpreds = read_vals(
      "abstention_experiments/catdog/"+model_dir+"/test_det_preds.txt.gz")
  orig_valid_detpreds = read_vals(
      "abstention_experiments/catdog/"+model_dir+"/valid_det_preds.txt.gz")
  orig_test_nondetpreds = np.array([
      read_vals("abstention_experiments/catdog/"
                +model_dir+"/test_nondet_preds_"+str(i)+".txt.gz")
      for i in range(100)]).transpose((1,0))
  
  for trial_idx in range(num_trials_per_model):
    
    print("\n###trial idx",trial_idx)
    
    rng = np.random.RandomState(trial_idx*100 + model_dir_idx*10)  
    bootstrap_valid_indices = rng.choice(
        a=list(range(len(orig_valid_labels))),
        size=len(orig_valid_labels), replace=True)
    indices_of_test_positives = [x[0] for x in enumerate(orig_test_labels)
                                 if x[1]==1]
    indices_of_test_negatives = [x[0] for x in enumerate(orig_test_labels)
                                 if x[1]==0]
    
    bootstrap_test_positive_indices = rng.choice(
        a=indices_of_test_positives,
        size=int(len(orig_valid_labels)/(3.0)),
        replace=True)
    bootstrap_test_negative_indices = rng.choice(
        a=indices_of_test_negatives,
        size=(len(orig_valid_labels)-len(bootstrap_test_positive_indices)),
        replace=True)
    bootstrap_test_indices =(list(bootstrap_test_negative_indices)+
                             list(bootstrap_test_positive_indices))
    
    bootstrap_valid_labels = orig_valid_labels[bootstrap_valid_indices]
    bootstrap_valid_detpreds = orig_valid_detpreds[bootstrap_valid_indices]
    bootstrap_test_labels = orig_test_labels[bootstrap_test_indices]
    bootstrap_test_detpreds = orig_test_detpreds[bootstrap_test_indices]
    bootstrap_test_nondetpreds = orig_test_nondetpreds[bootstrap_test_indices]
      
    #perform the calibration
    calib_func = calibfactory(valid_preacts=bootstrap_valid_detpreds,
                               valid_labels=bootstrap_valid_labels)
    calib_bootstrap_valid_detpreds = calib_func(
        preact=bootstrap_valid_detpreds)
    calib_bootstrap_test_detpreds = calib_func(
        preact=bootstrap_test_detpreds)
    calib_bootstrap_test_nondetpreds = (
      np.array([calib_func(x) for x in
                bootstrap_test_nondetpreds.transpose((1,0))
               ]).transpose((1,0)))

    adaptname_to_bootstrap_test_detpreds = OrderedDict()
    adaptname_to_bootstrap_test_nondetpreds = OrderedDict()
    #perform the adaptation
    for adaptname in adaptname_to_factory:  
      adaptation_func = adaptname_to_factory[adaptname](
          valid_labels=bootstrap_valid_labels,
          tofit_initial_posterior_probs=calib_bootstrap_test_detpreds,
          valid_posterior_probs=calib_bootstrap_valid_detpreds)
      adapted_bootstrap_test_detpreds = adaptation_func(
          unadapted_posterior_probs=calib_bootstrap_test_detpreds)
      adaptname_to_bootstrap_test_detpreds[adaptname] =\
        adapted_bootstrap_test_detpreds
      adapted_bootstrap_test_nondetpreds = np.array(
          [adaptation_func(x) for x in
           calib_bootstrap_test_nondetpreds.transpose((1,0))]).transpose((1,0))
      adaptname_to_bootstrap_test_nondetpreds[adaptname] =\
        adapted_bootstrap_test_nondetpreds
       
    for adaptname in adaptname_to_factory:
      #compute the baseline test perfs
      for metricname in metricname_to_computefunc:
        computefunc = metricname_to_computefunc[metricname]
        perf = computefunc(
            y_true=bootstrap_test_labels,
            y_score=adaptname_to_bootstrap_test_detpreds[adaptname])        
        metricname_to_adaptname_to_baselineperfs[
            metricname][adaptname].append(perf)
        print("Base perf",metricname,perf)
      #perform abstentions   
      for abstname in abstname_to_factory:
        print("\nOn method", abstname+":"+adaptname)
        abstention_func = abstname_to_factory[abstname](
            valid_labels=bootstrap_valid_labels,
            valid_posterior=calib_bootstrap_valid_detpreds)
        #get the abstention priorities, and sort the indices accordingly
        adapted_bootstrap_test_detpreds =\
          adaptname_to_bootstrap_test_detpreds[adaptname]
        var_adapted_bootstrap_test_nondetpreds = np.var(
            adaptname_to_bootstrap_test_nondetpreds[adaptname], axis=1)
        bootstrap_test_abstention_priorities = abstention_func(
            posterior_probs=adapted_bootstrap_test_detpreds,
            uncertainties=var_adapted_bootstrap_test_nondetpreds)
        sorted_indices = [
              x[0] for x in
              sorted(enumerate(bootstrap_test_abstention_priorities),
              key=lambda x: x[1])]

        for abstfrac in abstfracs:
          #get the post-abstention examples            
          num_to_retain = int(len(sorted_indices)*(1-abstfrac))
          retained_test_detpreds = adapted_bootstrap_test_detpreds[
              sorted_indices[:num_to_retain]]
          retained_test_labels =\
            bootstrap_test_labels[sorted_indices[:num_to_retain]]
          for metricname in metricname_to_computefunc:
            metric_compute_func = metricname_to_computefunc[metricname]
            perf = metric_compute_func(y_true=retained_test_labels,
                                       y_score=retained_test_detpreds)
            metricname_to_abstfrac_to_abstnadapt_to_perf[metricname]\
              [abstfrac][abstname+":"+adaptname].append(perf)
            print("Abstfrac",abstfrac,metricname,perf)


###On model dir record_1_model_wqD9U_preds

###trial idx 0
Base perf tpratfpr_0.01 0.34633853541416565
Base perf auroc 0.8996756471034724

On method mcwindowdelta_tpratfpr_30pc:EM


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Abstfrac 0.15 tpratfpr_0.01 0.6299126637554585
Abstfrac 0.15 auroc 0.8812001791781381
Abstfrac 0.3 tpratfpr_0.01 0.6332931242460796
Abstfrac 0.3 auroc 0.9233025585534484

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.435589519650655
Abstfrac 0.15 auroc 0.8607728236012187
Abstfrac 0.3 tpratfpr_0.01 0.5073746312684366
Abstfrac 0.3 auroc 0.8895801843501021

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4802021903959562
Abstfrac 0.15 auroc 0.9093160451633362
Abstfrac 0.3 tpratfpr_0.01 0.6036988110964333
Abstfrac 0.3 auroc 0.9095827447890654

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4559721011333915
Abstfrac 0.15 auroc 0.9009423903127187
Abstfrac 0.3 tpratfpr_0.01 0.455
Abstfrac 0.3 auroc 0.8963564516129032

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.3779697624190065
Abstfrac 0.15 auroc 0.9224246331527313
Abstfrac 0.3 tpratfpr_0.01 0.464
Abstfrac 0.3 auroc 0.9426646081871345

On method estwindowdelta_auroc_30pc:EM
Abstfrac

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Base perf tpratfpr_0.01 0.31812725090036015
Base perf auroc 0.907658354283525

On method mcwindowdelta_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5786026200873362
Abstfrac 0.15 auroc 0.90125670280791
Abstfrac 0.3 tpratfpr_0.01 0.5915985997666278
Abstfrac 0.3 auroc 0.9374192898967838

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4443231441048035
Abstfrac 0.15 auroc 0.8747614560057421
Abstfrac 0.3 tpratfpr_0.01 0.553623188405797
Abstfrac 0.3 auroc 0.9070491000051577

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4426644182124789
Abstfrac 0.15 auroc 0.917668848709267
Abstfrac 0.3 tpratfpr_0.01 0.6450381679389313
Abstfrac 0.3 auroc 0.9273672372637591

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4430379746835443
Abstfrac 0.15 auroc 0.9174998795437806
Abstfrac 0.3 tpratfpr_0.01 0.4059633027522936
Abstfrac 0.3 auroc 0.9040488687809902

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.3603411513859275
Abstfrac 0.15 auroc 0.930579

...

Abstfrac 0.15 tpratfpr_0.01 0.5906113537117904
Abstfrac 0.15 auroc 0.875029961256657
Abstfrac 0.3 tpratfpr_0.01 0.6387176325524044
Abstfrac 0.3 auroc 0.917585184009934

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4388646288209607
Abstfrac 0.15 auroc 0.858223169776525
Abstfrac 0.3 tpratfpr_0.01 0.5247524752475248
Abstfrac 0.3 auroc 0.8938817542948095

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4846894138232721
Abstfrac 0.15 auroc 0.8987972295223638
Abstfrac 0.3 tpratfpr_0.01 0.5981308411214953
Abstfrac 0.3 auroc 0.9047864133882133

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4094631483166515
Abstfrac 0.15 auroc 0.8884478229393502
Abstfrac 0.3 tpratfpr_0.01 0.38920454545454547
Abstfrac 0.3 auroc 0.88380609622271

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.38358103326256193
Abstfrac 0.15 auroc 0.9230456352101851
Abstfrac 0.3 tpratfpr_0.01 0.40966921119592875
Abstfrac 0.3 auroc 0.9439887460400467

On method estwindowdel

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Base perf tpratfpr_0.01 0.3403361344537815
Base perf auroc 0.8953034723187415

On method mcwindowdelta_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5829694323144105
Abstfrac 0.15 auroc 0.8681544258833823
Abstfrac 0.3 tpratfpr_0.01 0.6466916354556804
Abstfrac 0.3 auroc 0.9066353238518543

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4672489082969432
Abstfrac 0.15 auroc 0.8550068370605355
Abstfrac 0.3 tpratfpr_0.01 0.5504840940525588
Abstfrac 0.3 auroc 0.8884897729870587

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4800347222222222
Abstfrac 0.15 auroc 0.8938711018757622
Abstfrac 0.3 tpratfpr_0.01 0.5941255006675568
Abstfrac 0.3 auroc 0.8952957511748367

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4028776978417266
Abstfrac 0.15 auroc 0.8843897157608522
Abstfrac 0.3 tpratfpr_0.01 0.38461538461538464
Abstfrac 0.3 auroc 0.8781221966808701

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.37729196050775743
Abstfrac 0.15 auroc 0.

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Abstfrac 0.15 tpratfpr_0.01 0.5458515283842795
Abstfrac 0.15 auroc 0.8547543766355898
Abstfrac 0.3 tpratfpr_0.01 0.6069221260815822
Abstfrac 0.3 auroc 0.8995677116276891

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.37445414847161573
Abstfrac 0.15 auroc 0.8366669460867652
Abstfrac 0.3 tpratfpr_0.01 0.4270676691729323
Abstfrac 0.3 auroc 0.8581766585776611

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4591747146619842
Abstfrac 0.15 auroc 0.8833951519841374
Abstfrac 0.3 tpratfpr_0.01 0.5517241379310345
Abstfrac 0.3 auroc 0.8862913212562861

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.3939945404913558
Abstfrac 0.15 auroc 0.8735127199389884
Abstfrac 0.3 tpratfpr_0.01 0.46175637393767704
Abstfrac 0.3 auroc 0.8803527978892619

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.3551336146272855
Abstfrac 0.15 auroc 0.9137191974170292
Abstfrac 0.3 tpratfpr_0.01 0.3797996661101836
Abstfrac 0.3 auroc 0.9354542540492481

On method estwindo

In [4]:
file_out = "catdog_abstention_results.json"
dict_to_write = {
    "metricname_to_adaptname_to_baselineperfs":
     metricname_to_adaptname_to_baselineperfs,
    "metricname_to_abstfrac_to_abstnadapt_to_perf":
     metricname_to_abstfrac_to_abstnadapt_to_perf}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0